In [7]:
import sys
import os
from datetime import datetime
sys.path.append('..')

import pandas as pd
import numpy as np
from ml_models.anomaly_detector_v3 import AnomalyDetectorV3

print("🚀 JUNTA ANALYTICS - DETECTOR DE ANOMALÍAS V3")
print(f"Fecha: {datetime.now():%Y-%m-%d %H:%M}")
print("="*60)

🚀 JUNTA ANALYTICS - DETECTOR DE ANOMALÍAS V3
Fecha: 2025-05-25 21:19


In [8]:
# Cargar datos
DATA_DIR = "../data/processed"
individual_df = pd.read_csv(
    os.path.join(DATA_DIR, "individual_consumption.csv"),
    parse_dates=["created_at", "period_start", "period_end"],
)

print(f"📈 Total de lecturas: {len(individual_df):,}")
print(f"🏠 Medidores únicos: {individual_df['water_meter_id'].nunique():,}")
print(f"📅 Rango de fechas: {individual_df['period_start'].min().date()} → {individual_df['period_start'].max().date()}")

# Limpiar datos
train_df = individual_df[
    (individual_df['total_consumed'] >= 0) & 
    (individual_df['total_consumed'] <= 500) &  # Filtrar extremos
    (individual_df['days_billed'] > 0)
].copy()

print(f"\n🧹 Después de limpieza: {len(train_df):,} lecturas")

📈 Total de lecturas: 10,067
🏠 Medidores únicos: 362
📅 Rango de fechas: 2022-12-01 → 2025-04-01

🧹 Después de limpieza: 10,067 lecturas


In [9]:
# Entrenar modelo
detector = AnomalyDetectorV3(contamination=0.1, random_state=42)

print("🔄 Entrenando detector híbrido...")
metrics = detector.train(train_df)

print("\n📊 Métricas de entrenamiento:")
for key, value in metrics.items():
    print(f"  {key}: {value}")

🔄 Entrenando detector híbrido...

📊 Métricas de entrenamiento:
  samples_trained: 10067
  meters_trained: 362
  features_used: 10
  if_score_mean: 0.06769675073292407
  svm_score_mean: 12.594652082211844
  contamination: 0.1


In [10]:
# Seleccionar un medidor real para las pruebas
sample_meter = train_df['water_meter_id'].iloc[0]
meter_data = train_df[train_df['water_meter_id'] == sample_meter]
typical_consumption = meter_data['total_consumed'].median()

print(f"🏠 Medidor de prueba: {sample_meter}")
print(f"📊 Lecturas históricas: {len(meter_data)}")
print(f"💧 Consumo típico: {typical_consumption} unidades")
print(f"📈 Rango de consumo: {meter_data['total_consumed'].min()}-{meter_data['total_consumed'].max()}")

# Casos de prueba
test_cases = [
    {
        "name": "✅ Consumo Normal",
        "current_reading": 1000 + int(typical_consumption),
        "previous_reading": 1000,
        "days_billed": 30,
        "expected": False
    },
    {
        "name": "🚨 Consumo Alto (Fuga)",
        "current_reading": 1000 + int(typical_consumption * 5),
        "previous_reading": 1000,
        "days_billed": 30,
        "expected": True
    },
    {
        "name": "⚠️ Consumo Cero",
        "current_reading": 1000,
        "previous_reading": 1000,
        "days_billed": 30,
        "expected": True
    },
    {
        "name": "❌ Lectura Negativa",
        "current_reading": 980,
        "previous_reading": 1000,
        "days_billed": 30,
        "expected": True
    },
    {
        "name": "🔥 Consumo Extremo",
        "current_reading": 1000 + int(typical_consumption * 10),
        "previous_reading": 1000,
        "days_billed": 30,
        "expected": True
    },
    {
        "name": "📉 Consumo Muy Bajo",
        "current_reading": 1000 + max(1, int(typical_consumption * 0.1)),
        "previous_reading": 1000,
        "days_billed": 30,
        "expected": True
    }
]

print("\n🧪 Ejecutando casos de prueba...\n")

results = []
for i, case in enumerate(test_cases, 1):
    result = detector.detect_single_reading(
        water_meter_id=sample_meter,
        current_reading=case["current_reading"],
        previous_reading=case["previous_reading"],
        days_billed=case["days_billed"],
        historical_data=train_df
    )
    
    # Verificar si la predicción es correcta
    is_correct = result['is_anomaly'] == case['expected']
    status = "✅" if is_correct else "❌"
    
    print(f"{i}. {case['name']} {status}")
    print(f"   Consumo: {result['total_consumed']} unidades ({result['consumption_per_day']:.2f}/día)")
    print(f"   Detectado: {'🚨 ANOMALÍA' if result['is_anomaly'] else '✅ NORMAL'}")
    print(f"   Score: {result['score']:.3f} | Confianza: {result['confidence']:.3f}")
    print(f"   Razón: {result['reason']}")
    print(f"   Método: {result.get('detection_method', 'N/A')}")
    print()
    
    results.append({
        'case': case['name'],
        'expected': case['expected'],
        'detected': result['is_anomaly'],
        'correct': is_correct,
        'score': result['score'],
        'method': result.get('detection_method', 'N/A')
    })

# Calcular precisión
correct_predictions = sum(1 for r in results if r['correct'])
accuracy = correct_predictions / len(results)

print(f"📊 RESUMEN DE RESULTADOS:")
print(f"   Precisión: {accuracy:.1%} ({correct_predictions}/{len(results)})")
print(f"   Casos correctos: {correct_predictions}")
print(f"   Casos incorrectos: {len(results) - correct_predictions}")

🏠 Medidor de prueba: 479
📊 Lecturas históricas: 29
💧 Consumo típico: 4.0 unidades
📈 Rango de consumo: 1-12

🧪 Ejecutando casos de prueba...

1. ✅ Consumo Normal ✅
   Consumo: 4 unidades (0.13/día)
   Detectado: ✅ NORMAL
   Score: 0.000 | Confianza: 0.000
   Razón: Consumo normal dentro de parámetros esperados
   Método: none

2. 🚨 Consumo Alto (Fuga) ✅
   Consumo: 20 unidades (0.67/día)
   Detectado: 🚨 ANOMALÍA
   Score: 0.800 | Confianza: 0.850
   Razón: Anomalía estadística: 6.5σ del patrón del medidor (ratio: 4.8x)
   Método: statistical

3. ⚠️ Consumo Cero ✅
   Consumo: 0 unidades (0.00/día)
   Detectado: 🚨 ANOMALÍA
   Score: 0.800 | Confianza: 0.900
   Razón: Consumo cero: posible medidor roto o casa vacía
   Método: N/A

4. ❌ Lectura Negativa ✅
   Consumo: -20 unidades (0.00/día)
   Detectado: 🚨 ANOMALÍA
   Score: 1.000 | Confianza: 1.000
   Razón: Lectura negativa: lectura actual menor que anterior
   Método: N/A

5. 🔥 Consumo Extremo ✅
   Consumo: 40 unidades (1.33/día)
   Dete

In [11]:
# Guardar el modelo
model_path = "../data/models/anomaly_detector_v3.joblib"
detector.save(model_path)
print(f"💾 Modelo guardado en: {model_path}")

# Probar carga del modelo
detector_loaded = AnomalyDetectorV3()
detector_loaded.load(model_path)
print(f"📂 Modelo cargado exitosamente")
print(f"🔧 Características: {len(detector_loaded.feature_columns_)}")
print(f"📊 Medidores entrenados: {len(detector_loaded.meter_stats_) if detector_loaded.meter_stats_ is not None else 0}")

💾 Modelo guardado en: ../data/models/anomaly_detector_v3.joblib
📂 Modelo cargado exitosamente
🔧 Características: 10
📊 Medidores entrenados: 362
